### Closure Applications (Part 2)

#### Example 1

Давайте напишем небольшую функцию, которая может увеличить счетчик для нас — в Python у нас нет инкрементора (например, оператора ++ в Java или C++):

In [2]:
def counter(initial_value):
    # initial_value здесь — локальная переменная

    def inc(increment=1):
        nonlocal initial_value
        # initial_value здесь — нелокальная (захваченная) переменная
        initial_value += increment
        return initial_value

    return inc

In [3]:
counter1 = counter(0)

In [4]:
print(counter1(0))

0


In [5]:
print(counter1())

1


In [6]:
print(counter1())

2


In [7]:
print(counter1(8))

10


In [8]:
counter2 = counter(1000)

In [9]:
print(counter2(0))

1000


In [10]:
print(counter2(1))

1001


In [11]:
print(counter2())

1002


In [12]:
print(counter2(220))

1222


Как видите, каждое замыкание сохраняет ссылку на переменную **initial_value**, которая была создана при **вызове** функции **counter** - каждый раз, когда эта функция вызывалась, создавалась новая локальная переменная **initial_value** (со значением, назначенным из аргумента), и она становилась нелокальной (захваченной) переменной во внутренней области видимости.

#### Example 2

Давайте изменим этот пример, чтобы создать что-то, что может работать, и вести подсчет того, сколько раз мы запустили некоторую функцию.

In [13]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)

    return inner

In [14]:
def add(a, b):
    return a + b

In [15]:
counted_add = counter(add)

А свободные переменные:

In [16]:
counted_add.__code__.co_freevars

('cnt', 'fn')

Теперь мы можем вызвать функцию `counted_add`:

In [17]:
counted_add(1, 2)

add has been called 1 times


3

In [18]:
counted_add(2, 3)

add has been called 2 times


5

In [19]:
def mult(a, b, c):
    return a * b * c

In [20]:
counted_mult = counter(mult)

In [21]:
counted_mult(1, 2, 3)

mult has been called 1 times


6

In [22]:
counted_mult(2, 3, 4)

mult has been called 2 times


24

#### Example 3

Давайте сделаем еще один шаг вперед и сохраним имя функции и количество вызовов в глобальном словаре, а не просто будем постоянно его выводить.

In [35]:
counters = dict()

def counter(fn):
    cnt = 0  # изначально fn был запущен ноль раз

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is global
        return fn(*args, **kwargs)

    return inner

In [26]:
counted_add = counter(add)
counted_mult = counter(mult)

Обратите внимание, что `counters` — это **глобальная** переменная, а значит **не** свободная переменная:

In [27]:
counted_add.__code__.co_freevars

('cnt', 'fn')

In [28]:
counted_mult.__code__.co_freevars

('cnt', 'fn')

Теперь мы можем вызывать функции

In [29]:
counted_add(1, 2)

3

In [30]:
counted_add(2, 3)

5

In [31]:
counted_mult(1, 2, 'a')

'aa'

In [32]:
counted_mult(2, 3, 'b')

'bbbbbb'

In [33]:
counted_mult(1, 1, 'abc')

'abc'

In [34]:
print(counters)

{'add': 2, 'mult': 3}


Конечно, это предполагает, что мы сначала создадим глобальную переменную **counters** и убедимся, что мы даем ей соответствующее имя, поэтому вместо этого мы передадим ее в качестве аргумента функции **counter**:

In [36]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)

    return inner

In [33]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [34]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

Как видите, `counters` теперь является свободной переменной.

Теперь мы можем вызывать наши функции:

In [35]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

In [36]:
print(func_counters)

{'add': 5, 'mult': 10}


Конечно, нам не нужно присваивать «подсчитанной» версии наших функций новое имя — мы можем просто присвоить ей то же самое имя!

In [37]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [38]:
fact = counter(fact, func_counters)

In [39]:
fact(0)

1

In [40]:
fact(3)

6

In [41]:
fact(4)

24

In [42]:
print(func_counters)

{'add': 5, 'mult': 10, 'fact': 3}


Обратите внимание, как мы по сути **добавили** некоторую функциональность к нашей функции `fact`, не изменяя то, что функция `fact` фактически возвращает.

Это приводит нас прямо к нашей следующей теме: декораторы!

---